# 4. Modeling – Previsão de Churn com Regressão Logística

Nesta etapa aplicamos um modelo supervisionado de classificação
para prever a probabilidade de churn dos clientes.

O algoritmo escolhido foi Regressão Logística, amplamente utilizado
em problemas binários por sua interpretabilidade e eficiência.


In [0]:
df = spark.table("`crisp-dm`.gold_churn")

display(df)


## 4.1 Seleção de Variáveis

Para o modelo utilizaremos as métricas RFM:

- Recency
- Frequency
- Monetary

A variável target é:

- churn (1 = cliente inativo, 0 = ativo)


In [0]:
df_model = df.select(
    "recency",
    "frequency",
    "monetary",
    "churn"
)

display(df_model)


## 4.2 Vetorização das Features

O Spark ML requer que as variáveis explicativas sejam combinadas
em um único vetor numérico.


In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["recency", "frequency", "monetary"],
    outputCol="features"
)

df_vector = assembler.transform(df_model)

display(df_vector)


## 4.3 Divisão em Treino e Teste

Dividimos os dados em:

- 70% treino
- 30% teste

Garantindo avaliação fora da amostra.


In [0]:
train_df, test_df = df_vector.randomSplit([0.7, 0.3], seed=42)

print("Treino:", train_df.count())
print("Teste:", test_df.count())


## 4.4 Treinamento do Modelo

Aplicamos Regressão Logística para estimar a probabilidade
de churn com base nas variáveis RFM.


In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="churn"
)

model = lr.fit(train_df)


In [0]:
predictions = model.transform(test_df)

display(predictions)


## 4.5 Avaliação do Modelo

Avaliamos o desempenho utilizando:

- AUC (Area Under ROC Curve)
- Accuracy


In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="churn",
    metricName="areaUnderROC"
)

auc = evaluator.evaluate(predictions)

print("AUC:", auc)


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

accuracy_evaluator = MulticlassClassificationEvaluator(
    labelCol="churn",
    metricName="accuracy"
)

accuracy = accuracy_evaluator.evaluate(predictions)

print("Accuracy:", accuracy)


In [0]:
coefficients = model.coefficients
intercept = model.intercept

print("Coeficientes:", coefficients)
print("Intercept:", intercept)


A regressão logística apresentou AUC de 1.0,
indicando capacidade de discriminação entre clientes ativos e churn.

Os coeficientes indicam que:

- Recency tende a aumentar a probabilidade de churn
- Frequency tende a reduzir a probabilidade
- Monetary contribui para retenção dependendo do sinal estimado
